In [1]:
from pymongo import MongoClient
import pandas as pd
import pymongo
import folium

In [2]:
client = MongoClient('mongodb://localhost:27017/')
db = client.companies
data = db.data_companies_clean.find()

In [3]:
data_comp = pd.DataFrame(data)
display(data_comp.head())

,_id,alias_list,category_code,description,founded_year,lat,lng,main_office,total_money_raised
0,5d2633ac797d7972f3178d9e,,web,Life long learning content,2007,34.017606,-118.487267,"{'type': 'Point', 'coordinates': [-118.487267,...",$21M
1,5d2633ac797d7972f3178d9f,None,web,None,2007,37.269175,-119.306607,"{'type': 'Point', 'coordinates': [-119.306607,...",$100k
2,5d2633ac797d7972f3178da0,,web,Social network for desktop software,2007,52.374523,4.894862,"{'type': 'Point', 'coordinates': [4.8948623, 5...",$1M
3,5d2633ac797d7972f3178da1,,web,Video guide for hotels,2007,48.856667,2.350987,"{'type': 'Point', 'coordinates': [2.350987, 48...",$13.8M
4,5d2633ac797d7972f3178da2,None,web,None,2007,40.010492,-105.276843,"{'type': 'Point', 'coordinates': [-105.276843,...",$315k


In [4]:
# def findNear(df,radio_max_meters = 2000):
#     location = db.data_companies_clean.find({'main_office':{
#         '$near':{
#             '$geometry':df,
#             '$maxDistance': radio_max_meters,
#         }
#     }
#     })
#     df1 = pd.DataFrame(location)
#     return len(df1)
# near_offices = findNear(data_comp['main_office'][0])

In [5]:
def findNear(df, radio_max_meters = 20):
    lst =[]
    for i in range(len(df)):
        location = db.data_companies_clean.find({'main_office':{
                                                 '$near': {
                                                     '$geometry': df[i],
                                                     '$maxDistance': radio_max_meters,
                                                 }
                                                }
                                                })
        
        df2 = pd.DataFrame(location)
        lst.append(len(df2))    
    return lst

near_offices = findNear(data_comp['main_office'])
len(near_offices)

154

In [6]:
data_comp['Near offices'] = pd.DataFrame(near_offices)

In [17]:
data_comp.head()

,_id,alias_list,category_code,description,founded_year,lat,lng,main_office,total_money_raised,Near offices
0,5d2633ac797d7972f3178d9e,,web,Life long learning content,2007,34.017606,-118.487267,"{'type': 'Point', 'coordinates': [-118.487267,...",$21M,1
1,5d2633ac797d7972f3178d9f,None,web,None,2007,37.269175,-119.306607,"{'type': 'Point', 'coordinates': [-119.306607,...",$100k,2
2,5d2633ac797d7972f3178da0,,web,Social network for desktop software,2007,52.374523,4.894862,"{'type': 'Point', 'coordinates': [4.8948623, 5...",$1M,1
3,5d2633ac797d7972f3178da1,,web,Video guide for hotels,2007,48.856667,2.350987,"{'type': 'Point', 'coordinates': [2.350987, 48...",$13.8M,2
4,5d2633ac797d7972f3178da2,None,web,None,2007,40.010492,-105.276843,"{'type': 'Point', 'coordinates': [-105.276843,...",$315k,2


In [8]:
base_map = folium.Map(zoom_start=15)

In [37]:
a = list(zip(data_comp['lat'], data_comp['lng']))
for i in a:
    b = folium.Marker(location=[i[0],i[1]]).add_to(base_map) 
    
base_map